# VacationPy
----

#### Note
* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [47]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os
import json

# Import API key
from api_keys import g_key

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [48]:
# File to Load (Remember to Change These)
file_to_load = "Weather.csv"

# Read Purchasing File and store into Pandas data frame
weather_df = pd.read_csv(file_to_load)


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [49]:
 # Configure gmaps with API key
gmaps.configure(api_key=g_key)


 # Store 'Lat' and 'Lng' into  locations 
locations = weather_df[["Lat", "Lng"]].astype(float)

# Convert Poverty Rate to float and store
# HINT: be sure to handle NaN values
humidity = weather_df["Humidity"].astype(float)


In [50]:
fig = gmaps.figure()

heat_layer = gmaps.heatmap_layer(locations, weights=humidity, 
                                 dissipating=False, max_intensity=100,
                                 point_radius = 1)

fig.add_layer(heat_layer)

fig

Figure(layout=FigureLayout(height='420px'))

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [51]:
weather_df.drop(weather_df.loc[weather_df['Max Temp']<25].index, inplace=True)

weather_df.reset_index(drop=True)

,Unnamed: 0,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,teahupoo,-17.83,-149.27,28.00,58,40,2.60,PF,1592870420
1,7,morant bay,17.88,-76.41,32.00,59,20,8.70,JM,1592870421
2,9,avarua,-21.21,-159.78,25.00,69,22,6.20,CK,1592870421
3,15,albany,42.60,-73.97,31.67,54,35,1.36,US,1592870422
4,17,noumea,-22.28,166.46,25.00,53,28,1.50,NC,1592870195
...,...,...,...,...,...,...,...,...,...,...
183,584,nuevitas,21.55,-77.26,29.00,83,97,5.10,CU,1592870489
184,585,mae chan,20.15,99.85,26.67,94,40,1.00,TH,1592870489
185,591,bay saint louis,30.31,-89.33,28.00,69,40,5.70,US,1592870315
186,594,saint-prosper,46.21,-70.48,25.56,100,22,0.87,CA,1592870490


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [52]:
base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"


hotel_list = []
weather_df.loc[0]["Lat"]
for i in range(len(weather_df["Lat"])):
    lat = weather_df.iloc[i]['Lat']
    lng = weather_df.iloc[i]['Lng']
    params = {
        "location": f"{lat},{lng}",
        "radius": 5000,
        "types" : "lodging",
        "key": g_key
    }
    response = requests.get(base_url, params=params).json()
    
    try:
        #print(response['results'][0]['name'])
        take2 = (response['results'][0]['name'])
        hotel_list.append(take2)
        
    except:
        hotel_list.append('')
        #print("Missing value")






In [53]:
#lat = weather_df.iloc[0]['Lat']
#lng = weather_df.iloc[0]['Lng']
#params = {"location": f"{lat},{lng}", "radius": 5000, "types" : "hotel", "key": g_key}
#response = requests.get(base_url, params=params)
#print(json.dumps(response, indent=4, sort_keys=True))
#response.json()
hotel_list
weather_df["Hotel Name"] = hotel_list
weather_df = weather_df.dropna(how='any')
hotel_df = weather_df.copy()

hotel_df.reset_index(drop=True)


,Unnamed: 0,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date,Hotel Name
0,0,teahupoo,-17.83,-149.27,28.00,58,40,2.60,PF,1592870420,Vanira Lodge
1,7,morant bay,17.88,-76.41,32.00,59,20,8.70,JM,1592870421,Whispering Bamboo Cove Resort
2,9,avarua,-21.21,-159.78,25.00,69,22,6.20,CK,1592870421,Paradise Inn
3,15,albany,42.60,-73.97,31.67,54,35,1.36,US,1592870422,
4,17,noumea,-22.28,166.46,25.00,53,28,1.50,NC,1592870195,Hôtel Le Lagon
...,...,...,...,...,...,...,...,...,...,...,...
183,584,nuevitas,21.55,-77.26,29.00,83,97,5.10,CU,1592870489,Hotel Caonaba
184,585,mae chan,20.15,99.85,26.67,94,40,1.00,TH,1592870489,Kiang Dao Resort
185,591,bay saint louis,30.31,-89.33,28.00,69,40,5.70,US,1592870315,The Abode Guest House
186,594,saint-prosper,46.21,-70.48,25.56,100,22,0.87,CA,1592870490,


In [55]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Lat", "Lng"]]

In [58]:
# Add marker layer ontop of heat map
 # Create bank symbol layer
marker_layer = gmaps.symbol_layer(
    locations, fill_color='rgba(0, 150, 0, 0.4)',
    stroke_color='rgba(0, 0, 150, 0.4)', scale=2,
    info_box_content=[f"Hotel Amount: {hotel}" for hotel in hotel_list]
)


fig = gmaps.figure()
fig.add_layer(marker_layer)

fig

# Display figure
# Create a combined map
fig = gmaps.figure()

fig.add_layer(heat_layer)
fig.add_layer(marker_layer)

fig 

Figure(layout=FigureLayout(height='420px'))